<a href="https://colab.research.google.com/github/VittorioRossi/NLP-content-moderation/blob/main/dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_datasets

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
import pandas as pd
import numpy as np 
import os 
import tensorflow_dataset as tfds

In [ ]:
# directory
TMP_DIR = os.path.join('/tmp/data')

TRAIN_DIR = os.path.join(TMP_DIR, "train")
TEST_DIR = os.path.join(TMP_DIR, "test")

SEED = 42
BATCH_SIZE = 64


#creating folder structure
os.makedirs(TMP_DIR, exist_ok = True)
os.makedirs(TEST_DIR, exist_ok = True)
os.makedirs(TRAIN_DIR, exist_ok = True)


spam_data = pd.read_csv("/content/drive/MyDrive/Datasets/text-moderation/spam-train.csv")
spam_data = spam_data[["Tweet", "Type"]]
spam_data.columns = ['text', 'label']
spam_data.label = spam_data.label.map(lambda x: 0 if x == "Quality" else 2)

spam_data.label.value_counts()

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
toxic_train, toxic_test = tfds.load(name="WikipediaToxicitySubtypes", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

In [ ]:
toxic_train = pd.DataFrame({"text":toxic_train[0].numpy(), "label":toxic_train[1].numpy()})
toxic_test = pd.DataFrame({"text":toxic_test[0].numpy(), "label":toxic_test[1].numpy()})

In [ ]:
toxic_train.text = toxic_train.text.str.decode('utf-8') 
toxic_test.text = toxic_test.text.str.decode('utf-8') 

In [ ]:
final_data = pd.concat((toxic_train, toxic_test,spam_data))

In [ ]:
final_data.label.value_counts()

0.0    209619
1.0     21384
2.0      7445
Name: label, dtype: int64

In [ ]:
train_data = pd.concat((
    final_data[final_data.label == 0][:6_000], 
    final_data[final_data.label == 1][:6_000],
    final_data[final_data.label == 2][:6_000],
))

In [ ]:
test_data = final_data.drop(train_data.index)

In [ ]:
test_data.label.value_counts()

0.0    184617
1.0     14060
2.0      1314
Name: label, dtype: int64

In [ ]:
train_data.text = train_data.text
test_data.text = test_data.text

train_data.label = train_data.label.map(int)
test_data.label = test_data.label.map(int)

In [ ]:
train_data.to_csv("/content/drive/MyDrive/Datasets/text-moderation/merged_train.csv")
test_data.to_csv("/content/drive/MyDrive/Datasets/text-moderation/merged_test.csv")